In [ ]:
import os

model_dir = "./phi2-text2sql"  # Your output directory
checkpoints = [f for f in os.listdir(model_dir) if f.startswith("checkpoint")]
print("Available Checkpoints:", checkpoints)

Available Checkpoints: ['checkpoint-250', 'checkpoint-350']


In [ ]:
latest_checkpoint = sorted(checkpoints, key=lambda x: int(x.split("-")[-1]))[-1]  # Sort numerically
checkpoint_path = os.path.join(model_dir, latest_checkpoint)

print("Using checkpoint:", checkpoint_path)

from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the latest checkpoint
model = AutoModelForCausalLM.from_pretrained(checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
model.eval()  # Set to evaluation mode

Using checkpoint: ./phi2-text2sql/checkpoint-350


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=2560, out_features=2560, bias=True)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2560, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=2560, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=2560, out_features=2560, bias=True)
            (lora_dropout): ModuleDict(
              (default): Dropout(

In [ ]:
import torch

def generate_sql(query, model, tokenizer, max_length=200):
    inputs = tokenizer(query, return_tensors="pt").to("cpu")
    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(output[0], skip_special_tokens=True)

import re

def generate_sql(user_query, model, tokenizer):
    input_ids = tokenizer.encode(user_query, return_tensors="pt")
    output = model.generate(
        input_ids=input_ids,
        max_length=100,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(output[0], skip_special_tokens=True).strip()

# Example user query
user_query = """
## INPUT
Query: Find the total revenue generated in the year 2004.

## OUTPUT
SQL query:
"""

# Generate SQL from the model (Make sure `model` and `tokenizer` are defined)
generated_sql = generate_sql(user_query, model, tokenizer)

# Example output from the model (if already available)
generated_output = """
## INPUT
Query: Find the total revenue generated in the year 2004.

## OUTPUT
SQL query:
SELECT SUM(revenue) FROM orders WHERE order_date BETWEEN '2004-01-01' AND '2004-12-31';
"""

# Extract only the SQL query using regex
matches = re.findall(r"SQL query:\s*(SELECT.*?);", generated_output, re.DOTALL)

# Get the last (or first) SQL query found
cleaned_sql = matches[-1] + ";" if matches else "No valid SQL found."

print("Purified SQL Output:", cleaned_sql)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Purified SQL Output: SELECT SUM(revenue) FROM orders WHERE order_date BETWEEN '2004-01-01' AND '2004-12-31';


In [ ]:
! zip -r phi2-text2sql-finetuned.zip phi2-text2sql/

  adding: phi2-text2sql/ (stored 0%)
  adding: phi2-text2sql/checkpoint-250/ (stored 0%)
  adding: phi2-text2sql/checkpoint-250/merges.txt (deflated 53%)
  adding: phi2-text2sql/checkpoint-250/adapter_model.safetensors (deflated 7%)
  adding: phi2-text2sql/checkpoint-250/scheduler.pt (deflated 56%)
  adding: phi2-text2sql/checkpoint-250/special_tokens_map.json (deflated 75%)
  adding: phi2-text2sql/checkpoint-250/vocab.json (deflated 59%)
  adding: phi2-text2sql/checkpoint-250/tokenizer_config.json (deflated 94%)
  adding: phi2-text2sql/checkpoint-250/added_tokens.json (deflated 84%)
  adding: phi2-text2sql/checkpoint-250/README.md (deflated 66%)
  adding: phi2-text2sql/checkpoint-250/training_args.bin (deflated 51%)
  adding: phi2-text2sql/checkpoint-250/tokenizer.json (deflated 82%)
  adding: phi2-text2sql/checkpoint-250/adapter_config.json (deflated 55%)
  adding: phi2-text2sql/checkpoint-250/optimizer.pt (deflated 9%)
  adding: phi2-text2sql/checkpoint-250/rng_state.pth (deflated 2